In [1]:
! pip install pdfminer

In [ ]:
from google.colab import files
uploaded = files.upload()

In [2]:
fp = open('./annul_reports/2017_2330_20180605F04_20200708_220143.pdf','rb')

In [3]:
from pdfminer.pdfinterp import PDFPageInterpreter,PDFResourceManager
from pdfminer.converter import TextConverter,PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfpage import PDFPage

In [18]:
%%time
# pdf 轉 txt
# 获取pdf文档

# 创建一个与文档相关的解释器
parser = PDFParser(fp)

# pdf文档的对象，与解释器连接起来
doc = PDFDocument(parser=parser)
parser.set_document(doc=doc)

# 如果是加密pdf，则输入密码
# doc._initialize_password()

# 创建pdf资源管理器
resource = PDFResourceManager()

# 参数分析器
laparam=LAParams()

# 创建一个聚合器
device = PDFPageAggregator(resource,laparams=laparam)

# 创建pdf页面解释器
interpreter = PDFPageInterpreter(resource,device)

# 获取页面的集合
counter = 1
for page in PDFPage.get_pages(fp):
    # 使用页面解释器来读取
    interpreter.process_page(page)
    
    # 使用聚合器来获取内容
    layout = device.get_result()
    
    for out in layout:
        if hasattr(out,'get_text'):
            #print(out.get_text())
            counter += 1
            
            # 写入txt文件
            fw = open('pdfToTxt_part.txt','a',encoding="utf-8")
            fw.write(out.get_text())
            fw.close()
    if counter % 30 == 0:
        break

Wall time: 47.7 s


In [19]:
rf = open('pdfToTxt_part.txt', 'r', encoding='utf-8')  
lines = rf.readlines()
fw = open('pdfToTxt_part_paragraph.txt','w',encoding="utf-8")

lowLen = 13
paragraphLen = 20
long_line = ""
isLong = False


for line in lines:  
    if len(line) < paragraphLen:
        if isLong:
            # line = line.replace('\n', '').replace('\r', '')
            long_line += line
            fw.write(long_line)
            # print("writeLong", len(long_line),long_line )
          
            long_line = ""
            isLong = False
        else:
            if lowLen < len(line):
                 # print("writeShort",len(line),line)
                fw.write(line)  
    else:
        # print("isLong")
        line = line.replace('\n', '').replace('\r', '')
        long_line += line
        isLong = True
        
rf.close()
fw.close()

In [8]:
import os
import gensim
import jieba

In [20]:
rf = open('pdfToTxt_part_paragraph.txt', 'r', encoding='utf-8')  
wf = open('pdfToTxt_part_paragraph_cut.txt', 'w', encoding='utf-8')

In [21]:
lines = rf.readlines()
for line in lines:
    # print(line)
    document = line.replace('<p>', '').replace('</p>', '').replace('\n', '').replace('，', '').replace('。',
                                                                                                          '').replace(
        '：', '').replace('？', '').replace(' ', '').replace('"summarization":', '').replace('"article"', '').replace('{', '').replace('}', '').replace(':', '').replace('<Paragraph>', '')
    document_cut = jieba.cut(document, cut_all=False)
    result = ' '.join(document_cut)
    wf.write(result+'\n')
rf.close()
wf.close()

In [30]:
rf = open('pdfToTxt_part_paragraph.txt', 'r', encoding='utf-8') 
lines = rf.readlines()
document = {}
count = 0
for line in lines:
    document[count] = line
    count += 1
rf.close()
print(count)

682


In [34]:
sentences = gensim.models.doc2vec.TaggedLineDocument('pdfToTxt_part_paragraph.txt')  #  训练
model = gensim.models.Doc2Vec(sentences, vector_size=256, window=2)
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)
model.save('part_model.txt')
print(len(model.docvecs))  #  所有文章的向量

682


In [26]:
sentences2 = gensim.models.doc2vec.TaggedLineDocument('pdfToTxt_part_paragraph_cut.txt')  #  训练
model2 = gensim.models.Doc2Vec(sentences, vector_size=256, window=2)
model2.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)
model2.save('part_model_02.txt')
print(len(model2.docvecs))  #  所有文章的向量

682


In [28]:
temps = ["公司研發方向",
         "我想知道公司未來展望",
         "公司最近的股價",
         "我想知道公司今年的財政狀況",
         "銷售",
         "財務分析"]

query = []
for temp in temps:

    temp_ = jieba.cut(temp, cut_all=False)
    result = ' '.join(temp_)
    array = []
    get = ""
    for i in result:
        if i == ' ':
            array.append(get)
            get = ""
        else:
            get+= i
    array.append(get)
   # print(array)
    query.append(array)
print(query)

[['公司', '研發', '方向'], ['我', '想', '知道', '公司', '未來', '展望'], ['公司', '最近', '的', '股價'], ['我', '想', '知道', '公司', '今年', '的', '財政狀況'], ['銷售'], ['財務', '分析']]


In [35]:
for i in query:
    print(i)
    sims = model.docvecs.most_similar([model.infer_vector(i)],topn=3)  #输出和训练集中第一个文章最相近的文章
    # print(document[i])
    print(sims)
    for docid, sim in sims:
        print()
        print(document[docid])
        print("--------------------------")

['公司', '研發', '方向']
[(520, 0.20921361446380615), (149, 0.15752699971199036), (427, 0.1541743129491806)]

（一） 公司是否評估往來對象之誠信紀錄，並於其與往來交易對象簽訂之契約中明訂誠信行為條款？

--------------------------

●  解決新製程開發時的可靠性問題，改善並管理量產時的品質，提供客戶品質問題的解決方案，並提供先進材料與故障分析等服務，以確保公司產品品質及可靠性016

--------------------------

●  審閱會計師績效評量問卷結果

--------------------------
['我', '想', '知道', '公司', '未來', '展望']
[(168, 0.1844683587551117), (593, 0.17439189553260803), (639, 0.17156219482421875)]

前中華民國法務部法律事務司司長

--------------------------

28,050,000,000

--------------------------

5,341,120,243

--------------------------
['公司', '最近', '的', '股價']
[(132, 0.16414913535118103), (360, 0.1632072627544403), (289, 0.15698181092739105)]

4.  進一步拓展台積公司在新興市場及發展中市場的業務與服務。

--------------------------

ˇ ˇ ˇ ˇ ˇ ˇ ˇ

--------------------------

美國喬治亞理工學院材料工程博士

--------------------------
['我', '想', '知道', '公司', '今年', '的', '財政狀況']
[(194, 0.1856417953968048), (559, 0.17419381439685822), (416, 0.17355115711688995)]

前G3 Good Governance Group資深顧問（倫敦）倫敦

In [32]:
for i in query:
    print(i)
    sims = model2.docvecs.most_similar([model.infer_vector(i)],topn=3)  #输出和训练集中第一个文章最相近的文章
    # print(document[i])
    print(sims)
    for docid, sim in sims:
        print()
        print(document[docid])
        print("--------------------------")

['公司', '研發', '方向']
[(520, 0.20926041901111603), (149, 0.1555422842502594), (427, 0.15235382318496704)]

（一） 公司是否評估往來對象之誠信紀錄，並於其與往來交易對象簽訂之契約中明訂誠信行為條款？

--------------------------

●  解決新製程開發時的可靠性問題，改善並管理量產時的品質，提供客戶品質問題的解決方案，並提供先進材料與故障分析等服務，以確保公司產品品質及可靠性016

--------------------------

●  審閱會計師績效評量問卷結果

--------------------------
['我', '想', '知道', '公司', '未來', '展望']
[(168, 0.18446923792362213), (593, 0.1743934005498886), (639, 0.17156080901622772)]

前中華民國法務部法律事務司司長

--------------------------

28,050,000,000

--------------------------

5,341,120,243

--------------------------
['公司', '最近', '的', '股價']
[(132, 0.16422012448310852), (360, 0.16342902183532715), (289, 0.15714791417121887)]

4.  進一步拓展台積公司在新興市場及發展中市場的業務與服務。

--------------------------

ˇ ˇ ˇ ˇ ˇ ˇ ˇ

--------------------------

美國喬治亞理工學院材料工程博士

--------------------------
['我', '想', '知道', '公司', '今年', '的', '財政狀況']
[(194, 0.18600010871887207), (559, 0.17423534393310547), (416, 0.17332270741462708)]

前G3 Good Governance Group資深顧問（倫敦）